<div align = "center">

# **Tarea 2: Conteo Aproximado**

</div>

## Librerias

In [84]:
import numpy as np
import numba
from numba import jit, prange
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.special import logsumexp
import pandas as pd
from tqdm import tqdm
import warnings
import os
import time
warnings.filterwarnings('ignore')

# Configuración de visualización
sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

# Configuración de semilla para reproducibilidad
np.random.seed(42)

# Crear directorio de resultados si no existe
os.makedirs('../results', exist_ok=True)
print("Directorio de resultados creado/verificado: ../results")

Directorio de resultados creado/verificado: ../results


## Funciones

### Parámetros según Teorema 9.1

In [85]:
# ============================================================================
# CÁLCULO DE PARÁMETROS SEGÚN TEOREMA 9.1
# ============================================================================

def calculate_n_samples(K, epsilon):
    """
    Calcula el número de simulaciones (muestras) necesarias según Teorema 9.1.
    
    Según el teorema, cada factor Yi se obtiene usando no más de:
        n_samples = 48 * d² * k³ / ε²
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice K×K
    epsilon : float
        Precisión deseada (ε)
    
    Retorna:
    --------
    n_samples : int
        Número de muestras (simulaciones) por ratio Yi
    
    Ejemplo:
    --------
    Para K=3, ε=0.1:
        k = 2*3*(3-1) = 12 aristas
        d = 4 (grado máximo en lattice)
        n_samples = 48 * 16 * 1728 / 0.01 = 132,710,400
    """
    # Para lattice K×K con bordes libres
    k = 2 * K * (K - 1)  # Número de aristas
    d = 4                 # Grado máximo (cada vértice tiene a lo más 4 vecinos)
    
    # Fórmula del teorema
    n_samples = (48 * d**2 * k**3) / (epsilon**2)
    
    return int(np.ceil(n_samples))


def calculate_n_steps_per_sample(K, q, epsilon):
    """
    Calcula el número de pasos del Gibbs sampler por simulación según Teorema 9.1.
    
    Por la ecuación (76) del teorema, cada simulación requiere no más de:
        n_steps = k * [2*log(k) + log(ε⁻¹) + log(8)] / log(q/(q-1)) + 1
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice K×K
    q : int
        Número de colores
    epsilon : float
        Precisión deseada (ε)
    
    Retorna:
    --------
    n_steps : int
        Número de pasos del Gibbs sampler por muestra
    
    Restricción:
    ------------
    El teorema requiere q > 2d* donde d* = 4 para lattice, entonces q > 8.
    Sin embargo, el algoritmo puede ejecutarse con q < 8 aunque sin garantías teóricas.
    
    Ejemplo:
    --------
    Para K=3, q=10, ε=0.1:
        k = 12
        numerador = 2*log(12) + log(10) + log(8) ≈ 9.05
        denominador = log(10/9) ≈ 0.105
        n_steps = 12 * (9.05/0.105 + 1) ≈ 1,047
    """
    # Para lattice K×K con bordes libres
    k = 2 * K * (K - 1)  # Número de aristas
    
    # Validar que q > 1 (necesario para que log(q/(q-1)) esté definido)
    if q <= 1:
        raise ValueError(f"q debe ser > 1, recibido q={q}")
    
    # Calcular según fórmula del teorema
    numerator = 2 * np.log(k) + np.log(1/epsilon) + np.log(8)
    denominator = np.log(q / (q - 1))
    
    # Evitar división por cero o valores negativos
    if denominator <= 0:
        raise ValueError(f"log(q/(q-1)) debe ser > 0, q={q}")
    
    n_steps = k * (numerator / denominator + 1)
    
    return int(np.ceil(n_steps))


def print_theorem_parameters(K, q, epsilon):
    """
    Imprime los parámetros calculados según el Teorema 9.1 y estadísticas de tiempo.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    """
    k = 2 * K * (K - 1)
    d = 4
    
    n_samples = calculate_n_samples(K, epsilon)
    n_steps = calculate_n_steps_per_sample(K, q, epsilon)
    
    # Estadísticas
    total_steps_per_ratio = n_samples * n_steps
    total_steps_all_ratios = k * total_steps_per_ratio
    
    print(f"\n{'='*70}")
    print(f"PARÁMETROS SEGÚN TEOREMA 9.1")
    print(f"{'='*70}")
    print(f"Lattice: {K}×{K}")
    print(f"Número de colores (q): {q}")
    print(f"Precisión (ε): {epsilon}")
    print(f"Número de aristas (k): {k}")
    print(f"Grado máximo (d): {d}")
    
    if q <= 2*d:
        print(f"\n⚠️  ADVERTENCIA: Teorema requiere q > 2d = {2*d}, pero q = {q}")
        print(f"   El algoritmo se ejecutará pero sin garantías teóricas.")
    
    print(f"\n{'─'*70}")
    print(f"PARÁMETROS CALCULADOS:")
    print(f"{'─'*70}")
    print(f"Muestras por ratio (n_samples): {n_samples:,}")
    print(f"Pasos por muestra (n_steps): {n_steps:,}")
    print(f"\n{'─'*70}")
    print(f"ESTADÍSTICAS COMPUTACIONALES:")
    print(f"{'─'*70}")
    print(f"Pasos totales por ratio: {total_steps_per_ratio:,}")
    print(f"Pasos totales para {k} ratios: {total_steps_all_ratios:,}")
    
    # Estimar tiempo (asumiendo ~1M pasos/segundo con JIT)
    estimated_seconds = total_steps_all_ratios / 1_000_000
    estimated_minutes = estimated_seconds / 60
    estimated_hours = estimated_minutes / 60
    
    print(f"\nTiempo estimado (1M pasos/seg):")
    if estimated_hours > 1:
        print(f"  ~{estimated_hours:.2f} horas")
    elif estimated_minutes > 1:
        print(f"  ~{estimated_minutes:.2f} minutos")
    else:
        print(f"  ~{estimated_seconds:.2f} segundos")
    
    print(f"{'='*70}\n")
    
    return n_samples, n_steps

In [ ]:
# ============================================================================
# CÁLCULO DE PARÁMETROS
# ============================================================================

def calc_n_samples(K, q, epsilon):
    """
    Número mínimo de muestras por ratio.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    
    Retorna:
    --------
    n_samples : int
    """
    # Fórmula simple basada en precisión
    return int(10_000 / epsilon)


def calc_n_steps_per_sample(K, q, epsilon):
    """
    Número mínimo de pasos del Gibbs sampler por muestra.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    
    Retorna:
    --------
    n_steps : int
    """
    # Tiempo de mezcla: τ ≈ K² log(K)
    N = K * K
    tau_mix = int(N * np.log(N + 1))
    
    # Ajustar por q (más colores = más rápido)
    n_steps = int(tau_mix / np.log(q + 1))
    
    return max(100, n_steps)

In [ ]:
# ============================================================================
# CONTEO DE q-COLORACIONES - VERSIÓN LIMPIA
# ============================================================================

def count_colorings(K, q, epsilon, max_steps_per_ratio):
    """
    Cuenta q-coloraciones en lattice K×K usando método telescópico.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    epsilon : float
        Precisión deseada
    max_steps_per_ratio : int
        Máximo de pasos totales del Gibbs sampler por ratio
    
    Retorna:
    --------
    result : dict
        - 'K': tamaño lattice
        - 'q': número de colores
        - 'epsilon': precisión usada
        - 'log_count': log(Z_{G,q})
        - 'count': Z_{G,q}
        - 'n_samples': muestras objetivo
        - 'n_steps': pasos por muestra
        - 'avg_ratio': ratio promedio de los k ratios
        - 'total_time': tiempo en segundos
    """
    # Calcular parámetros
    n_samples = calc_n_samples(K, q, epsilon)
    n_steps = calc_n_steps_per_sample(K, q, epsilon)
    
    # Crear aristas
    N = K * K
    all_edges = create_lattice_edges(K)
    k = len(all_edges)  # k = 2K(K-1)
    
    # Z_0 = q^(K²)
    log_Z_0 = N * np.log(q)
    
    # Estimar ratios
    log_product = 0.0
    ratios = []
    
    start_time = time.time()
    
    for i in range(1, k + 1):
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        edges_i = all_edges[:i]
        
        ratio, _, _ = estimate_ratio_telescopic_with_limit(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps,
            max_steps_total=max_steps_per_ratio,
            verbose=False
        )
        
        ratio_safe = max(ratio, 1e-300)
        log_product += np.log(ratio_safe)
        ratios.append(ratio)
    
    total_time = time.time() - start_time
    
    # Resultado final
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    return {
        'K': K,
        'q': q,
        'epsilon': epsilon,
        'log_count': log_count,
        'count': count,
        'n_samples': n_samples,
        'n_steps': n_steps,
        'avg_ratio': np.mean(ratios),
        'total_time': total_time
    }

In [87]:
# ============================================================================
# CONTEO TELESCÓPICO CON PARÁMETROS PRÁCTICOS
# ============================================================================

def count_colorings_telescopic_practical(K, q, precision_level='medium', verbose=True):
    """
    Cuenta q-coloraciones usando parámetros PRÁCTICOS (no teóricos).
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice K×K
    q : int
        Número de colores
    precision_level : str
        Nivel de precisión: 'low', 'medium', 'high', 'very_high'
    verbose : bool
        Si True, imprime información detallada
    
    Retorna:
    --------
    count : float
        Estimación del número de q-coloraciones válidas
    log_count : float
        Logaritmo de la estimación
    results_df : DataFrame
        DataFrame con información detallada de cada ratio
    """
    # Calcular parámetros prácticos
    if verbose:
        n_samples, n_steps, max_steps = print_practical_parameters(K, q, precision_level)
    else:
        n_samples, n_steps, max_steps = calculate_practical_parameters(K, q, precision_level)
    
    # Número de vértices y aristas
    N_vertices = K * K
    all_edges = create_lattice_edges(K)
    k = len(all_edges)
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"INICIANDO CONTEO TELESCÓPICO (PARÁMETROS PRÁCTICOS)")
        print(f"{'='*70}\n")
    
    # Z_0 = q^(K^2)
    log_Z_0 = N_vertices * np.log(q)
    
    # Almacenar resultados
    results = []
    log_product = 0.0
    
    start_total = time.time()
    
    # Estimar cada ratio
    for i in range(1, k + 1):
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        edges_i = all_edges[:i]
        
        if verbose and (i == 1 or i % 3 == 0 or i == k):
            print(f"{'─'*70}")
            print(f"Ratio {i}/{k}: estimando r̂_{i} = Z_{i} / Z_{i-1}")
            print(f"{'─'*70}")
        
        start_ratio = time.time()
        
        ratio, samples_collected, steps_executed = estimate_ratio_telescopic_with_limit(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps,
            max_steps_total=max_steps,
            verbose=False  # No verbose interno para no saturar output
        )
        
        time_elapsed = time.time() - start_ratio
        
        ratio_safe = max(ratio, 1e-300)
        log_ratio = np.log(ratio_safe)
        log_product += log_ratio
        
        results.append({
            'i': i,
            'ratio': ratio,
            'log_ratio': log_ratio,
            'log_product_acum': log_product,
            'samples_requested': n_samples,
            'samples_collected': samples_collected,
            'steps_executed': steps_executed,
            'time_seconds': time_elapsed,
            'limit_reached': samples_collected < n_samples
        })
        
        if verbose and (i == 1 or i % 3 == 0 or i == k):
            limit_msg = " [LÍMITE]" if samples_collected < n_samples else ""
            print(f"  r̂_{i} = {ratio:.6f}{limit_msg}")
            print(f"  log(r̂_{i}) = {log_ratio:.4f}")
            print(f"  Muestras: {samples_collected:,}/{n_samples:,}")
            print(f"  Tiempo: {time_elapsed:.2f}s")
            print(f"  log_product_acum = {log_product:.4f}\n")
    
    # Resultado final
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    total_time = time.time() - start_total
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"RESULTADO FINAL")
        print(f"{'='*70}")
        print(f"  log(Z_0) = {log_Z_0:.4f}")
        print(f"  log(producto) = {log_product:.4f}")
        print(f"  log(Z_{{G,q}}) = {log_count:.4f}")
        print(f"  Z_{{G,q}} ≈ {count:.6e}")
        print(f"\n  Tiempo total: {total_time:.2f}s ({total_time/60:.2f} min)")
        
        total_samples_collected = sum(r['samples_collected'] for r in results)
        total_steps_executed = sum(r['steps_executed'] for r in results)
        num_limits_reached = sum(1 for r in results if r['limit_reached'])
        
        print(f"  Total de muestras colectadas: {total_samples_collected:,}")
        print(f"  Total de pasos ejecutados: {total_steps_executed:,}")
        if num_limits_reached > 0:
            print(f"  Ratios que alcanzaron el límite: {num_limits_reached}/{k}")
        print(f"{'='*70}\n")
    
    results_df = pd.DataFrame(results)
    
    return count, log_count, results_df

In [ ]:
# ============================================================================
# FUNCIÓN PARA DETERMINAR PASOS MÁXIMOS POR (K, q)
# ============================================================================

def calculate_max_steps_per_ratio(K, q):
    """
    Calcula el número máximo de pasos por ratio basado en K y q.
    
    Heurística:
    - K pequeño (3-5): más pasos para mejor precisión
    - K grande (15-20): menos pasos por ratio (hay más ratios)
    - q pequeño (2-4): más pasos (problema más restringido)
    - q grande (10-15): menos pasos (mezcla más rápida)
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    
    Retorna:
    --------
    max_steps : int
        Número máximo de pasos por ratio
    """
    # Base: escalado por tamaño de la lattice
    k = 2 * K * (K - 1)  # Número de aristas
    N = K * K            # Número de vértices
    
    # Factor base: más pasos para lattices pequeñas
    if K <= 5:
        base_steps = 20_000_000  # 20M
    elif K <= 10:
        base_steps = 10_000_000  # 10M
    elif K <= 15:
        base_steps = 5_000_000   # 5M
    else:
        base_steps = 2_000_000   # 2M
    
    # Ajustar por número de colores
    if q <= 3:
        color_factor = 1.5   # Más restringido, más pasos
    elif q <= 6:
        color_factor = 1.0
    elif q <= 10:
        color_factor = 0.7
    else:
        color_factor = 0.5   # Menos restringido, menos pasos
    
    max_steps = int(base_steps * color_factor)
    
    return max_steps

In [ ]:
# ============================================================================
# VERSIÓN SILENCIOSA PARA EXPERIMENTOS MASIVOS
# ============================================================================

def count_colorings_quiet(K, q, max_steps_per_ratio=None):
    """
    Versión SILENCIOSA del conteo telescópico para experimentos masivos.
    
    Solo reporta resultado final, sin logs intermedios.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    max_steps_per_ratio : int, optional
        Límite de pasos por ratio. Si None, se calcula automáticamente.
    
    Retorna:
    --------
    result : dict
        Diccionario con:
        - 'K': tamaño de la lattice
        - 'q': número de colores
        - 'log_count': logaritmo del conteo
        - 'count': conteo (puede ser inf)
        - 'total_time': tiempo total en segundos
        - 'total_samples': muestras totales colectadas
        - 'total_steps': pasos totales ejecutados
        - 'n_ratios': número de ratios estimados
        - 'avg_ratio': ratio promedio
        - 'min_ratio': ratio mínimo
        - 'max_ratio': ratio máximo
    """
    # Calcular parámetros automáticamente
    n_samples, n_steps, _ = calculate_practical_parameters(K, q, precision_level='medium')
    
    if max_steps_per_ratio is None:
        max_steps_per_ratio = calculate_max_steps_per_ratio(K, q)
    
    # Crear aristas
    N_vertices = K * K
    all_edges = create_lattice_edges(K)
    k = len(all_edges)
    
    # Z_0
    log_Z_0 = N_vertices * np.log(q)
    
    # Estimar ratios
    log_product = 0.0
    ratios = []
    total_samples_collected = 0
    total_steps_executed = 0
    
    start_total = time.time()
    
    for i in range(1, k + 1):
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        edges_i = all_edges[:i]
        
        ratio, samples_collected, steps_executed = estimate_ratio_telescopic_with_limit(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps,
            max_steps_total=max_steps_per_ratio,
            verbose=False
        )
        
        ratio_safe = max(ratio, 1e-300)
        log_ratio = np.log(ratio_safe)
        log_product += log_ratio
        
        ratios.append(ratio)
        total_samples_collected += samples_collected
        total_steps_executed += steps_executed
    
    total_time = time.time() - start_total
    
    # Resultado final
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    result = {
        'K': K,
        'q': q,
        'log_count': log_count,
        'count': count,
        'total_time': total_time,
        'total_samples': total_samples_collected,
        'total_steps': total_steps_executed,
        'n_ratios': k,
        'avg_ratio': np.mean(ratios),
        'min_ratio': np.min(ratios),
        'max_ratio': np.max(ratios),
        'max_steps_per_ratio': max_steps_per_ratio,
        'n_samples_target': n_samples,
        'n_steps_per_sample': n_steps
    }
    
    return result

In [ ]:
# ============================================================================
# SCRIPT PARA EJECUTAR TODOS LOS EXPERIMENTOS
# ============================================================================

def run_all_experiments(K_min=3, K_max=20, q_min=2, q_max=15, 
                       epsilon=1.0,
                       max_steps_func=None,
                       results_dir='../results', 
                       checkpoint_file='checkpoint.csv'):
    """
    Ejecuta todos los experimentos para K ∈ [K_min, K_max] y q ∈ [q_min, q_max].
    
    Parámetros:
    -----------
    K_min, K_max : int
        Rango de tamaños de lattice
    q_min, q_max : int
        Rango de número de colores
    epsilon : float
        Precisión para todos los experimentos
    max_steps_func : callable, optional
        Función que recibe (K, q) y retorna max_steps_per_ratio.
        Si None, usa valores por defecto.
    results_dir : str
        Directorio para guardar resultados
    checkpoint_file : str
        Archivo CSV para checkpoints
    """
    # Función por defecto para max_steps
    if max_steps_func is None:
        def max_steps_func(K, q):
            if K <= 5:
                base = 20_000_000
            elif K <= 10:
                base = 10_000_000
            elif K <= 15:
                base = 5_000_000
            else:
                base = 2_000_000
            if q <= 3:
                return int(base * 1.5)
            elif q <= 6:
                return base
            else:
                return int(base * 0.7)
    
    os.makedirs(results_dir, exist_ok=True)
    checkpoint_path = os.path.join(results_dir, checkpoint_file)
    
    # Cargar previos
    completed = set()
    results = []
    if os.path.exists(checkpoint_path):
        df_prev = pd.read_csv(checkpoint_path)
        completed = set(zip(df_prev['K'], df_prev['q']))
        results = df_prev.to_dict('records')
        print(f"📂 {len(completed)} experimentos ya completados\n")
    
    # Calcular total
    K_values = list(range(K_min, K_max + 1))
    q_values = list(range(q_min, q_max + 1))
    total = len(K_values) * len(q_values)
    remaining = total - len(completed)
    
    print(f"{'='*70}")
    print(f"K ∈ [{K_min},{K_max}], q ∈ [{q_min},{q_max}], ε={epsilon}")
    print(f"Total: {total}, Completados: {len(completed)}, Pendientes: {remaining}")
    print(f"{'='*70}\n")
    
    if remaining == 0:
        print("✅ Completado")
        return pd.read_csv(checkpoint_path)
    
    count = len(completed)
    start_total = time.time()
    
    # Ejecutar experimentos
    for K in K_values:
        for q in q_values:
            if (K, q) in completed:
                continue
            
            count += 1
            max_steps = max_steps_func(K, q)
            
            print(f"[{count}/{total}] K={K:2d} q={q:2d} max_steps={max_steps:,} ... ", 
                  end='', flush=True)
            
            try:
                result = count_colorings(K, q, epsilon, max_steps)
                
                print(f"✓ log(Z)={result['log_count']:7.2f} "
                      f"Z≈{result['count']:.2e} "
                      f"t={result['total_time']:5.1f}s "
                      f"r̄={result['avg_ratio']:.3f}")
                
                results.append(result)
                pd.DataFrame(results).to_csv(checkpoint_path, index=False)
                
            except Exception as e:
                print(f"❌ {str(e)}")
    
    total_time = time.time() - start_total
    
    print(f"\n{'='*70}")
    print(f"Completado: {count}/{total}")
    print(f"Tiempo: {total_time:.1f}s ({total_time/60:.1f} min)")
    print(f"Guardado: {checkpoint_path}")
    print(f"{'='*70}\n")
    
    df_final = pd.DataFrame(results)
    final_path = os.path.join(results_dir, 'results_complete.csv')
    df_final.to_csv(final_path, index=False)
    
    return df_final

## Ejecución de Todos los Experimentos

In [ ]:
# ============================================================================
# PRUEBA PEQUEÑA: K ∈ [3,5], q ∈ [2,5]
# ============================================================================

df_test = run_all_experiments(
    K_min=3, 
    K_max=5, 
    q_min=2, 
    q_max=5,
    epsilon=1.0,
    results_dir='../results',
    checkpoint_file='test.csv'
)

In [ ]:
# ============================================================================
# EJECUCIÓN COMPLETA: K ∈ [3,20], q ∈ [2,15]
# ============================================================================

df_all = run_all_experiments(
    K_min=3, 
    K_max=20, 
    q_min=2, 
    q_max=15,
    epsilon=1.0,  # Precisión
    results_dir='../results',
    checkpoint_file='colorings_full.csv'
)

In [88]:
# ============================================================================
# ESTIMACIÓN DE RATIOS CON LÍMITE DE PASOS
# ============================================================================

def estimate_ratio_telescopic_with_limit(K, edges_i_minus_1, edges_i, q, 
                                         n_samples, n_steps_per_sample,
                                         max_steps_total=None,
                                         verbose=False):
    """
    Estima la razón r_i = Z_i / Z_{i-1} usando el método telescópico CON LÍMITE DE PASOS.
    
    Genera UNA muestra válida cada vez ejecutando n_steps_per_sample del Gibbs sampler.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    edges_i_minus_1 : ndarray de forma (i-1, 4)
        Aristas del grafo G_{i-1}
    edges_i : ndarray de forma (i, 4)
        Aristas del grafo G_i
    q : int
        Número de colores
    n_samples : int
        Número MÁXIMO de muestras para estimar el ratio
    n_steps_per_sample : int
        Pasos del Gibbs sampler para generar CADA muestra
    max_steps_total : int, optional
        Número MÁXIMO TOTAL de pasos del Gibbs sampler permitidos.
        Si None, se permite ejecutar todos los n_samples.
        Si se alcanza este límite, se usan las muestras colectadas hasta ese momento.
    verbose : bool
        Si True, imprime progreso cada cierto número de muestras
    
    Retorna:
    --------
    ratio : float
        Estimación del ratio r_i = Z_i / Z_{i-1}
    samples_collected : int
        Número real de muestras colectadas
    steps_executed : int
        Número total de pasos del Gibbs sampler ejecutados
    """
    # Inicializar coloración aleatoria de la lattice K×K
    coloring = np.random.randint(0, q, size=(K, K))
    
    # Colectar muestras y contar cuántas son válidas para G_i
    valid_count = 0
    samples_collected = 0
    steps_executed = 0
    
    print_interval = max(1, n_samples // 10)  # Imprimir cada 10% si verbose=True
    
    for sample_idx in range(n_samples):
        # Verificar si excedemos el límite de pasos
        if max_steps_total is not None:
            if steps_executed + n_steps_per_sample > max_steps_total:
                if verbose:
                    print(f"    ⚠️  Límite de pasos alcanzado: {steps_executed:,}/{max_steps_total:,} pasos. "
                          f"Muestras colectadas: {samples_collected:,}/{n_samples:,}")
                break
        
        # Generar UNA muestra ejecutando n_steps_per_sample del Gibbs sampler
        run_gibbs_sampler_partial(coloring, edges_i_minus_1, q, n_steps_per_sample)
        steps_executed += n_steps_per_sample
        
        # Verificar si la coloración también es válida para G_i
        if is_valid_coloring(coloring, edges_i):
            valid_count += 1
        
        samples_collected += 1
        
        # Imprimir progreso si verbose
        if verbose and (sample_idx + 1) % print_interval == 0:
            ratio_so_far = valid_count / samples_collected if samples_collected > 0 else 0
            print(f"    Progreso: {samples_collected:,}/{n_samples:,} muestras, "
                  f"ratio={ratio_so_far:.4f}, "
                  f"pasos={steps_executed:,}")
    
    # Estimar ratio
    if samples_collected == 0:
        ratio = 0.0
    else:
        ratio = valid_count / samples_collected
    
    return ratio, samples_collected, steps_executed

In [89]:
# ============================================================================
# CONTEO TELESCÓPICO CON PARÁMETROS DEL TEOREMA 9.1
# ============================================================================

def count_colorings_telescopic_theorem(K, q, epsilon=0.1, 
                                       max_steps_per_ratio=None,
                                       verbose=True):
    """
    Cuenta q-coloraciones usando el método telescópico con parámetros del Teorema 9.1.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice K×K
    q : int
        Número de colores
    epsilon : float
        Precisión deseada (ε) según el teorema
    max_steps_per_ratio : int, optional
        Número MÁXIMO de pasos del Gibbs sampler para estimar cada ratio Yi.
        Si None, se usan todos los pasos que requiera el teorema.
        Si se especifica, limita el cómputo aunque no complete todas las muestras.
    verbose : bool
        Si True, imprime información detallada del progreso
    
    Retorna:
    --------
    count : float
        Estimación del número de q-coloraciones válidas
    log_count : float
        Logaritmo de la estimación
    results_df : DataFrame
        DataFrame con información detallada de cada ratio
    """
    # Calcular parámetros según Teorema 9.1
    if verbose:
        n_samples, n_steps = print_theorem_parameters(K, q, epsilon)
    else:
        n_samples = calculate_n_samples(K, epsilon)
        n_steps = calculate_n_steps_per_sample(K, q, epsilon)
    
    # Número de vértices y aristas
    N_vertices = K * K
    all_edges = create_lattice_edges(K)
    k = len(all_edges)  # k = 2*K*(K-1)
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"INICIANDO CONTEO TELESCÓPICO")
        print(f"{'='*70}")
        if max_steps_per_ratio is not None:
            print(f"Límite de pasos por ratio: {max_steps_per_ratio:,}")
            max_samples_possible = max_steps_per_ratio // n_steps
            print(f"Muestras máximas con este límite: {max_samples_possible:,}/{n_samples:,}")
        else:
            print(f"Sin límite de pasos (se ejecutarán todos los requeridos)")
        print(f"{'='*70}\n")
    
    # Z_0 = q^(K^2) (grafo sin aristas)
    log_Z_0 = N_vertices * np.log(q)
    
    # Almacenar resultados de cada ratio
    results = []
    log_product = 0.0
    
    start_total = time.time()
    
    # Estimar cada ratio r_i = Z_i / Z_{i-1}
    for i in range(1, k + 1):
        # Grafo G_{i-1}: primeras i-1 aristas
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        
        # Grafo G_i: primeras i aristas  
        edges_i = all_edges[:i]
        
        if verbose:
            print(f"{'─'*70}")
            print(f"Ratio {i}/{k}: estimando r̂_{i} = Z_{i} / Z_{i-1}")
            print(f"{'─'*70}")
        
        start_ratio = time.time()
        
        # Estimar ratio con límite de pasos
        ratio, samples_collected, steps_executed = estimate_ratio_telescopic_with_limit(
            K, edges_i_minus_1, edges_i, q,
            n_samples, n_steps,
            max_steps_total=max_steps_per_ratio,
            verbose=verbose
        )
        
        time_elapsed = time.time() - start_ratio
        
        # Evitar log(0)
        ratio_safe = max(ratio, 1e-300)
        log_ratio = np.log(ratio_safe)
        log_product += log_ratio
        
        # Guardar resultados
        results.append({
            'i': i,
            'ratio': ratio,
            'log_ratio': log_ratio,
            'log_product_acum': log_product,
            'samples_requested': n_samples,
            'samples_collected': samples_collected,
            'steps_executed': steps_executed,
            'time_seconds': time_elapsed,
            'limit_reached': samples_collected < n_samples
        })
        
        if verbose:
            limit_msg = " [LÍMITE]" if samples_collected < n_samples else ""
            print(f"  r̂_{i} = {ratio:.6f}{limit_msg}")
            print(f"  log(r̂_{i}) = {log_ratio:.4f}")
            print(f"  Muestras: {samples_collected:,}/{n_samples:,}")
            print(f"  Pasos: {steps_executed:,}")
            print(f"  Tiempo: {time_elapsed:.2f}s")
            print(f"  log_product_acum = {log_product:.4f}\n")
    
    # Z_k ≈ Z_0 * producto de ratios
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf
    
    total_time = time.time() - start_total
    
    if verbose:
        print(f"\n{'='*70}")
        print(f"RESULTADO FINAL")
        print(f"{'='*70}")
        print(f"  log(Z_0) = {log_Z_0:.4f}")
        print(f"  log(producto) = {log_product:.4f}")
        print(f"  log(Z_{{G,q}}) = {log_count:.4f}")
        print(f"  Z_{{G,q}} ≈ {count:.6e}")
        print(f"\n  Tiempo total: {total_time:.2f}s ({total_time/60:.2f} min)")
        
        # Estadísticas de muestras
        total_samples_collected = sum(r['samples_collected'] for r in results)
        total_steps_executed = sum(r['steps_executed'] for r in results)
        num_limits_reached = sum(1 for r in results if r['limit_reached'])
        
        print(f"  Total de muestras colectadas: {total_samples_collected:,}")
        print(f"  Total de pasos ejecutados: {total_steps_executed:,}")
        if num_limits_reached > 0:
            print(f"  Ratios que alcanzaron el límite: {num_limits_reached}/{k}")
        print(f"{'='*70}\n")
    
    # Crear DataFrame con resultados
    results_df = pd.DataFrame(results)
    
    return count, log_count, results_df

In [90]:
# ============================================================================
# REPRESENTACIÓN DE LA LATTICE K×K
# ============================================================================

def create_lattice_edges(K):
    """
    Crea la lista completa de aristas de una lattice K×K (grilla rectangular).
    
    REPRESENTACIÓN:
    - Vértices: coordenadas (x, y) donde x, y ∈ {0, 1, ..., K-1}
    - Aristas: 4-tuplas (x₁, y₁, x₂, y₂) representando conexión entre (x₁,y₁) y (x₂,y₂)
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    
    Retorna:
    --------
    edges : ndarray de forma (2*K*(K-1), 4)
        Lista de aristas, cada fila es [x₁, y₁, x₂, y₂]
        
    Nota:
    -----
    Total de aristas en lattice K×K con bordes libres:
    - Aristas horizontales: K × (K-1)  (conectan (x,y) con (x+1,y))
    - Aristas verticales: K × (K-1)    (conectan (x,y) con (x,y+1))
    - Total: 2K(K-1)
    
    Ejemplo K=3:
        (0,0)—(1,0)—(2,0)
          |     |     |
        (0,1)—(1,1)—(2,1)
          |     |     |
        (0,2)—(1,2)—(2,2)
    
    Aristas horizontales: 
        (0,0,1,0), (1,0,2,0), (0,1,1,1), (1,1,2,1), (0,2,1,2), (1,2,2,2)  → 6
    Aristas verticales:   
        (0,0,0,1), (0,1,0,2), (1,0,1,1), (1,1,1,2), (2,0,2,1), (2,1,2,2)  → 6
    Total: 12 aristas
    """
    edges = []
    
    # Aristas HORIZONTALES: (x,y) → (x+1,y)
    for y in range(K):
        for x in range(K - 1):
            edges.append((x, y, x+1, y))
    
    # Aristas VERTICALES: (x,y) → (x,y+1)
    for y in range(K - 1):
        for x in range(K):
            edges.append((x, y, x, y+1))
    
    # Convertir a numpy array
    edges_array = np.array(edges, dtype=np.int64)
    
    return edges_array


# ============================================================================
# VALIDACIÓN DE COLORACIONES
# ============================================================================

@jit(nopython=True)
def is_valid_coloring(coloring, edges):
    """
    Verifica si una coloración es válida para un grafo dado.
    
    Parámetros:
    -----------
    coloring : ndarray de forma (K, K)
        Coloración de la lattice (matriz K×K)
    edges : ndarray de forma (n_edges, 4)
        Lista de aristas del grafo, cada fila es [x₁, y₁, x₂, y₂]
    
    Retorna:
    --------
    valid : bool
        True si la coloración es válida (no hay aristas con endpoints del mismo color)
    """
    for edge_idx in range(len(edges)):
        x1 = edges[edge_idx, 0]
        y1 = edges[edge_idx, 1]
        x2 = edges[edge_idx, 2]
        y2 = edges[edge_idx, 3]
        
        if coloring[x1, y1] == coloring[x2, y2]:
            return False
    
    return True


@jit(nopython=True)
def get_neighbor_colors(x, y, coloring, edges):
    """
    Obtiene los colores de los vecinos de (x,y) en el grafo parcial.
    
    Parámetros:
    -----------
    x, y : int
        Coordenadas del vértice
    coloring : ndarray de forma (K, K)
        Coloración actual de la lattice
    edges : ndarray de forma (n_edges, 4)
        Lista de aristas del grafo parcial
    
    Retorna:
    --------
    neighbor_colors : set
        Conjunto de colores usados por los vecinos conectados por aristas en 'edges'
    """
    neighbor_colors = set()
    
    for edge_idx in range(len(edges)):
        x1 = edges[edge_idx, 0]
        y1 = edges[edge_idx, 1]
        x2 = edges[edge_idx, 2]
        y2 = edges[edge_idx, 3]
        
        # Si (x,y) está en esta arista, el otro endpoint es su vecino
        if x1 == x and y1 == y:
            neighbor_colors.add(coloring[x2, y2])
        elif x2 == x and y2 == y:
            neighbor_colors.add(coloring[x1, y1])
    
    return neighbor_colors


# ============================================================================
# GIBBS SAMPLER PARA GRAFO PARCIAL
# ============================================================================

@jit(nopython=True)
def gibbs_step_partial(coloring, edges, q):
    """
    Realiza un paso del Gibbs sampler para un grafo parcial.
    
    IMPORTANTE: Muestrea sobre TODA la lattice K×K, pero solo considera
    restricciones de las aristas que están en 'edges' (el grafo parcial Gᵢ).
    
    Parámetros:
    -----------
    coloring : ndarray de forma (K, K)
        Coloración actual de la lattice (se modifica in-place)
    edges : ndarray de forma (n_edges, 4)
        Aristas del grafo parcial Gᵢ
    q : int
        Número de colores
    
    Modifica coloring in-place.
    """
    K = coloring.shape[0]
    
    # 1. Seleccionar vértice aleatorio de TODA la lattice K×K
    x = np.random.randint(0, K)
    y = np.random.randint(0, K)
    
    # 2. Encontrar colores de vecinos (solo los conectados por aristas en 'edges')
    neighbor_colors = get_neighbor_colors(x, y, coloring, edges)
    
    # 3. Crear lista de colores VÁLIDOS
    valid_colors = []
    for c in range(q):
        if c not in neighbor_colors:
            valid_colors.append(c)
    
    # 4. Si hay colores válidos, muestrear uniformemente
    if len(valid_colors) > 0:
        idx = np.random.randint(0, len(valid_colors))
        coloring[x, y] = valid_colors[idx]
    # Si no hay colores válidos (no debería pasar con q suficientemente grande),
    # mantener el color actual


@jit(nopython=True)
def run_gibbs_sampler_partial(coloring, edges, q, n_steps):
    """
    Ejecuta múltiples pasos del Gibbs sampler para un grafo parcial.
    
    Parámetros:
    -----------
    coloring : ndarray de forma (K, K)
        Coloración inicial de la lattice (se modifica in-place)
    edges : ndarray de forma (n_edges, 4)
        Aristas del grafo parcial
    q : int
        Número de colores
    n_steps : int
        Número de pasos a ejecutar
    """
    for _ in range(n_steps):
        gibbs_step_partial(coloring, edges, q)


# ============================================================================
# ESTIMACIÓN DE RATIOS TELESCÓPICOS
# ============================================================================

@jit(nopython=True)
def estimate_ratio_telescopic(K, edges_i_minus_1, edges_i, q, 
                               n_samples, n_burn_in, n_spacing):
    """
    Estima la razón r_i = Z_i / Z_{i-1} usando el método telescópico.
    
    Muestrea coloraciones válidas de G_{i-1} (lattice con aristas edges_{i-1})
    y cuenta qué proporción también son válidas para G_i.
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    edges_i_minus_1 : ndarray de forma (i-1, 4)
        Aristas del grafo G_{i-1}
    edges_i : ndarray de forma (i, 4)
        Aristas del grafo G_i
    q : int
        Número de colores
    n_samples : int
        Número de muestras para estimar el ratio
    n_burn_in : int
        Pasos de burn-in
    n_spacing : int
        Espaciamiento entre muestras
    
    Retorna:
    --------
    ratio : float
        Estimación del ratio r_i = Z_i / Z_{i-1}
    """
    # Inicializar coloración aleatoria de la lattice K×K
    coloring = np.random.randint(0, q, size=(K, K))
    
    # Burn-in: alcanzar equilibrio muestreando de G_{i-1}
    run_gibbs_sampler_partial(coloring, edges_i_minus_1, q, n_burn_in)
    
    # Colectar muestras y contar cuántas son válidas para G_i
    valid_count = 0
    
    for sample_idx in range(n_samples):
        # Espaciar muestras para reducir autocorrelación
        run_gibbs_sampler_partial(coloring, edges_i_minus_1, q, n_spacing)
        
        # Verificar si la coloración también es válida para G_i
        if is_valid_coloring(coloring, edges_i):
            valid_count += 1
    
    # Estimar ratio
    ratio = valid_count / n_samples
    
    return ratio


# ============================================================================
# MÉTODO TELESCÓPICO COMPLETO
# ============================================================================

def count_colorings_telescopic(K, q, n_samples=1000, n_burn_in=None, n_spacing=None):
    """
    Cuenta aproximadamente el número de q-coloraciones válidas en una lattice K×K
    usando el método telescópico con MCMC (según teoría páginas 10-12).
    
    Parámetros:
    -----------
    K : int
        Tamaño de la lattice
    q : int
        Número de colores
    n_samples : int
        Número de muestras por ratio
    n_burn_in : int
        Pasos de burn-in (por defecto: 1000 * K^2)
    n_spacing : int
        Espaciamiento entre muestras (por defecto: 100 * K)
    
    Retorna:
    --------
    count : float
        Estimación del número de q-coloraciones válidas
    log_count : float
        Logaritmo de la estimación
    """
    # Parámetros por defecto
    if n_burn_in is None:
        n_burn_in = 1000 * K * K
    if n_spacing is None:
        n_spacing = 100 * K
    
    # Número de vértices
    N_vertices = K * K
    
    # Crear lista completa de aristas de la lattice (con bordes libres)
    all_edges = create_lattice_edges(K)
    l = len(all_edges)  # Número total de aristas = 2*K*(K-1)
    
    print(f"\n{'='*70}")
    print(f"Conteo telescópico para K={K}, q={q}")
    print(f"{'='*70}")
    print(f"Tamaño de lattice: {K}×{K}")
    print(f"Número de vértices: {N_vertices}")
    print(f"Número de aristas: {l} (esperado: {2*K*(K-1)})")
    print(f"Número de ratios a estimar: {l}")
    print(f"Muestras por ratio: {n_samples}")
    print(f"Burn-in: {n_burn_in}, Spacing: {n_spacing}")
    print(f"{'='*70}\n")
    
    # Z_0 = q^(K^2) (grafo sin aristas, todas las coloraciones son válidas)
    log_Z_0 = N_vertices * np.log(q)
    
    # Producto telescópico en log-escala
    log_product = 0.0
    
    # Estimar cada ratio r_i = Z_i / Z_{i-1}
    for i in range(1, l + 1):
        # Grafo G_{i-1}: primeras i-1 aristas
        edges_i_minus_1 = all_edges[:i-1] if i > 1 else np.array([], dtype=np.int64).reshape(0, 4)
        
        # Grafo G_i: primeras i aristas
        edges_i = all_edges[:i]
        
        # Estimar ratio
        ratio = estimate_ratio_telescopic(K, edges_i_minus_1, edges_i, q,
                                          n_samples, n_burn_in, n_spacing)
        
        # Evitar log(0) añadiendo un pequeño epsilon
        ratio = max(ratio, 1e-100)
        log_ratio = np.log(ratio)
        log_product += log_ratio
        
        if i % 5 == 0 or i == l or i == 1:
            print(f"  Arista {i}/{l}: r̂_{i} = {ratio:.6f}, "
                  f"log(r̂_{i}) = {log_ratio:.4f}, "
                  f"log_product_acum = {log_product:.4f}")
    
    # Z_l ≈ Z_0 * producto de ratios
    log_count = log_Z_0 + log_product
    count = np.exp(log_count) if log_count < 700 else np.inf  # Evitar overflow
    
    print(f"\n{'='*70}")
    print(f"  log(Z_0) = {log_Z_0:.4f}")
    print(f"  log(producto) = {log_product:.4f}")
    print(f"  log(Z_{{G,q}}) = {log_count:.4f}")
    print(f"  Z_{{G,q}} ≈ {count:.6e}")
    print(f"{'='*70}\n")
    
    return count, log_count

## Experimentos completos